In [1]:
# ! pip install agent-framework-azure-ai -U

In [ ]:

from agent_framework.azure import AzureAIAgentClient
from agent_framework import ChatAgent, HostedWebSearchTool

from azure.ai.projects.aio import AIProjectClient
from azure.ai.agents.models import BingGroundingTool
from azure.identity.aio import AzureCliCredential

In [3]:

import os
import base64
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
AgentName = "Search-Agent"
AgentInstructions = "You are an AI assistant that helps people find information."

In [6]:
conn_id = os.environ["BING_CONNECTION_ID"]  # Ensure the BING_CONNECTION_NAME environment variable is set

# Initialize the Bing Grounding tool
bing = BingGroundingTool(connection_id=conn_id)

In [7]:
bing

In [8]:
async with (
        AzureCliCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as client,
    ):

        created_agent = await client.agents.create_agent(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"], 
            instructions=AgentInstructions,
            name=AgentName,
            tools=bing.definitions
        )
        chat_client=AzureAIAgentClient(project_client=client, agent_id=created_agent.id)

        async with ChatAgent(
                # passing in the client is optional here, so if you take the agent_id from the portal
                # you can use it directly without the two lines above.
                chat_client=chat_client,
                tools=HostedWebSearchTool()
            ) as agent:
            message = "Could you please describe the workshop according to the link https://github.com/kinfey/GHCAgentWorkshop?"
            # response = agent.run_stream(message)
            first_result = await agent.run(message)
            
            print(f"Agent: {first_result.text}")

Agent: The GHCAgentWorkshop on GitHub is a workshop designed to help participants create GitHub Copilot applications with zero coding, leveraging the AI-powered GitHub Copilot Agent. The workshop introduces users to the agent mode of GitHub Copilot Chat that allows direct project creation from requirements. It is structured in a hands-on way, consisting of multiple experiments to explore the capabilities of GitHub Copilot agents, from model functions to application building and deployment.

The workshop includes:
- Preparations such as having a GitHub account, forking and cloning the repository, and setting up Python and Visual Studio Code with relevant plugins.
- Experiment 1 (RAG Application): Understanding GitHub Copilot model capabilities, working with the agent mode, creating a front-end Copilot application, and calling GitHub models through the agent.
- Experiment 2 (Stock Analysis): Generating simulated data with the Copilot agent, building a stock UI, and developing both front-